In [1]:
from google.colab import drive

ModuleNotFoundError: No module named 'google.colab'

In [0]:
drive.mount('/content/drive')

## **Importing required Libraries**

In [0]:
# importing the libraries

import pandas as pd
import numpy as np
import shutil,os


In [0]:

# for reading and displaying images

from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD


## **Data Engineering**

In [0]:
# loading dataset
train = pd.read_csv('/content/drive/My Drive/Data/TrainSet.csv', index_col='id')
val= pd.read_csv('/content/drive/My Drive/Data/validationSet.csv',index_col='id')
# sample_submission = pd.read_csv('sample_submission_I5njJSF.csv')
train.head()

In [0]:
#Reshaping train images into 64x64 and creating the train_target
import PIL
from PIL import Image
train_target=[]

train_img_directory='/content/drive/My Drive/Data/Train_images/'
train_new_img_dir='/content/drive/My Drive/Data/Train_new_images/'
for subdir,dirs,files in os.walk(train_img_directory):
    for file in files:
      im= Image.open(subdir+file)
      im=im.resize((64,64),PIL.Image.ANTIALIAS)
      im.save(train_new_img_dir+file)

for subdir,dirs,files in os.walk(train_new_img_dir):
    for file in files:
      # print(file)
      train_target.append(train.loc[file])
 

In [0]:
#Reshaping validation images into 64x64 and creating the validation_target
val_target=[]

val_img_directory='/content/drive/My Drive/Data/Val_images/'
val_new_img_dir='/content/drive/My Drive/Data/Val_new_images/'
for subdir,dirs,files in os.walk(val_img_directory):
    for file in files:
      im= Image.open(subdir+file)
      im=im.resize((64,64),PIL.Image.ANTIALIAS)
      im.save(val_new_img_dir+file)

for subdir,dirs,files in os.walk(val_new_img_dir):
    for file in files:
      # print(file)
     val_target.append(val.loc[file])


In [0]:
# loading training images
train_img = []
train_directory='/content/drive/My Drive/Data/Train_new_images/'
for subdir,dirs,files in os.walk(train_directory):
    for file in files:
        img = imread(subdir+file, as_gray=True)
        # normalizing the pixel values
        img /= 255.0
        # converting the type of pixel to float 32
        img = img.astype('float32')
        # appending the image into the list
        train_img.append(img)

# converting the list to numpy array
train_x = np.array(train_img)

# defining the target
train_y = np.array(train_target)

train_x.shape
train_y.shape


(188, 1)

In [0]:
train_x.shape

In [0]:
# loading Validation images
val_img = []
val_img_directory='/content/drive/My Drive/Data/Val_new_images/'
for subdir,dirs,files in os.walk(val_img_directory):
    for file in files:
        img = imread(subdir+file, as_gray=True)
        # normalizing the pixel values
        img /= 255.0
        # converting the type of pixel to float 32
        img = img.astype('float32')
        # appending the image into the list
        val_img.append(img)

# converting the list to numpy array
val_x = np.array(val_img)

# defining the target
val_y = np.array(val_target)

val_x.shape

(47, 64, 64)

In [0]:
# visualizing images

i = 0

plt.figure(figsize=(10,10))
plt.subplot(221), plt.imshow(train_x[i], cmap='gray')
plt.subplot(222), plt.imshow(train_x[i+25], cmap='gray')
plt.subplot(223), plt.imshow(train_x[i+50], cmap='gray')
plt.subplot(224), plt.imshow(train_x[i+75], cmap='gray')

### **Converting Images to Torch Format**

In [0]:
# # converting training images into torch format
train_x = train_x.reshape(188, 1, 64, 64)
train_x  = torch.from_numpy(train_x)

# converting the target into torch format
train_y = train_y.astype(int);
train_y = torch.from_numpy(train_y)

# shape of training data
train_x.shape, train_y.shape

# converting validation images into torch format
val_x = val_x.reshape(47, 1, 64, 64)
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int);
val_y = torch.from_numpy(val_y)

# shape of validation data
val_x.shape, val_y.shape


(torch.Size([47, 1, 64, 64]), torch.Size([47, 1]))

In [0]:
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)

## **Defining Convolution Neural Network**

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Net(Module):
    def __init__(self):
      super(Net, self).__init__()
      #we define convolutional layers 
      self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3, stride= 1, padding = 1)
      self.bn1 = nn.BatchNorm2d(32)
      self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1)
      self.bn2 = nn.BatchNorm2d(64)
      self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride=1, padding = 1)
      self.bn3 = nn.BatchNorm2d(128)

      #2 fully connected layers to transform the output of the convolution layers to the final output
      self.fc1 = nn.Linear(in_features = 8*8*128, out_features = 128)
      self.fcbn1 = nn.BatchNorm1d(128)
      self.fc2 = nn.Linear(in_features = 128, out_features = 2)       
      self.dropout_rate = 0.2

    def forward(self, s):
      #we apply the convolution layers, followed by batch normalisation, 
      #maxpool and relu x 3
      s = self.bn1(self.conv1(s))        # batch_size x 32 x 64 x 64
      s = F.relu(F.max_pool2d(s, 2))     # batch_size x 32 x 32 x 32
      s = self.bn2(self.conv2(s))        # batch_size x 64 x 32 x 32
      s = F.relu(F.max_pool2d(s, 2))     # batch_size x 64 x 16 x 16
      s = self.bn3(self.conv3(s))        # batch_size x 128 x 16 x 16
      s = F.relu(F.max_pool2d(s, 2))     # batch_size x 128 x 8 x 8

      #flatten the output for each image
      s = s.view(-1, 8*8*128)  # batch_size x 8*8*128

      #apply 2 fully connected layers with dropout
      s = F.dropout(F.relu(self.fcbn1(self.fc1(s))), 
      p=self.dropout_rate, training=self.training)    # batch_size x 128
      s = self.fc2(s)                                     # batch_size x 6
      return F.log_softmax(s, dim=1)

### **Creating CNN model**

In [0]:
from torch.optim import Adam
# Check if gpu support is available
cuda_avail = torch.cuda.is_available()

# Create model, optimizer and loss function
model = Net()

#if cuda is available, move the model to the GPU
if cuda_avail:
    model.cuda()


#Define the optimizer and loss function
optimizer = Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
print(model)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=8192, out_features=128, bias=True)
  (fcbn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)


In [0]:
def train(epoch):

    model.train()
    tr_loss = 0
    # getting the training set
    x_train, y_train = Variable(train_x), Variable(train_y)

    # getting the validation set
    x_val, y_val = Variable(val_x), Variable(val_y)

    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()

    # prediction for training and validation set
    output_train = model(x_train)
    output_val = model(x_val)

    # computing the training and validation loss
    loss_train = loss_fn(output_train, torch.max(y_train, 1)[1])
    loss_val = loss_fn(output_val, torch.max(y_val, 1)[1])
    # train_losses.append(loss_train)
    # val_losses.append(loss_val)

   # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_train)
    

In [0]:
# defining the number of epochs
n_epochs = 50

# empty list to store training losses
train_losses = []

# empty list to store validation losses
val_losses = []

# training the model
for epoch in range(n_epochs):
    train(epoch)

Epoch :  1 	 loss : tensor(0.5396, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  3 	 loss : tensor(0.3830, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  5 	 loss : tensor(0.2961, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  7 	 loss : tensor(0.2304, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  9 	 loss : tensor(0.1803, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  11 	 loss : tensor(0.1448, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  13 	 loss : tensor(0.1214, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  15 	 loss : tensor(0.0999, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  17 	 loss : tensor(0.0857, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  19 	 loss : tensor(0.0736, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  21 	 loss : tensor(0.0597, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  23 	 loss : tensor(0.0568, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch :  25 	 loss : tensor(0.0492, device='c

In [0]:
# val_losses=[0.,2.5364e-09,1.0145e-08,1.2682e-08,7.6091e-09,4.0582e-08,5.0727e-09,5.0727e-09,2.5364e-09,7.6091e-09,4.5655e-08,7.6091e-09,1.5979e-07,2.5364e-08,2.5364e-09,5.0727e-08,2.5364e-09,0.,5.0727e-09,2.5364e-09,1.6740e-07,1.2682e-08,5.0727e-09,5.0727e-09,0.]
# train_losses=[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,6.3409e-10,0.,6.3409e-10,6.3409e-10,0.]

## **Prediction**

In [0]:
# prediction for validation set
with torch.no_grad():
    softmax = model(val_x.cuda())
    softmax=softmax.cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)
accuracy_score(val_y, predictions)

0.574468085106383